In [2]:
!pip install seqeval
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from datasets import load_dataset

#raw_datasets = load_dataset('conll2003')
#raw_datasets = load_dataset("universal_dependencies", "te_mtg")

raw_datasets = load_dataset("anishka/Te_En_Syn_dataset")

raw_datasets

/home/admin123/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'upos', 'xpos'],
        num_rows: 1059
    })
    validation: Dataset({
        features: ['idx', 'text', 'upos', 'xpos'],
        num_rows: 138
    })
    test: Dataset({
        features: ['idx', 'text', 'upos', 'xpos'],
        num_rows: 144
    })
})

In [4]:
raw_datasets["train"][0]["text"]

['vāḍu', 'puli', 'caṁpina', 'maniṣi', '.']

In [5]:
ner_feature = raw_datasets["train"].features["upos"]
ner_feature

Sequence(feature=ClassLabel(names=['NOUN', 'PUNCT', 'ADP', 'NUM', 'SYM', 'SCONJ', 'ADJ', 'PART', 'DET', 'CCONJ', 'PROPN', 'PRON', 'X', '_', 'ADV', 'INTJ', 'VERB', 'AUX'], id=None), length=-1, id=None)

In [6]:
label_names = ner_feature.feature.names
label_names

['NOUN',
 'PUNCT',
 'ADP',
 'NUM',
 'SYM',
 'SCONJ',
 'ADJ',
 'PART',
 'DET',
 'CCONJ',
 'PROPN',
 'PRON',
 'X',
 '_',
 'ADV',
 'INTJ',
 'VERB',
 'AUX']

In [7]:
words = raw_datasets["train"][0]["text"]
labels = raw_datasets["train"][0]["upos"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)
    
print(line1)
print(line2)

vāḍu puli caṁpina maniṣi .     
PRON NOUN VERB    NOUN   PUNCT 


In [8]:
chunk_feature = raw_datasets["train"].features["upos"]
chunk_labels = chunk_feature.feature.names
words = raw_datasets["train"][0]["text"]
labels = raw_datasets["train"][0]["upos"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = chunk_labels[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)
    
print(line1)
print(line2)

vāḍu puli caṁpina maniṣi .     
PRON NOUN VERB    NOUN   PUNCT 


In [9]:
from transformers import AutoTokenizer

#model_checkpoint = "bert-base-cased"
#checkpoint = "xlm-roberta-base"
model_checkpoint = "anishka/xlm-roberta-base-te-transliterate"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
inputs = tokenizer(raw_datasets["train"][0]["text"], is_split_into_words=True)
inputs.tokens()

['<s>',
 '▁',
 'vā',
 'ḍ',
 'u',
 '▁pul',
 'i',
 '▁ca',
 'ṁ',
 'pina',
 '▁mani',
 'ṣ',
 'i',
 '▁',
 '.',
 '</s>']

In [11]:
inputs.word_ids()

[None, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, None]

In [12]:
labels = raw_datasets["train"][0]["upos"]
labels

[11, 0, 16, 0, 1]

In [13]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            new_labels.append(label)
    return new_labels

In [14]:
labels = raw_datasets["train"][0]["upos"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[11, 0, 16, 0, 1]
[-100, 11, 12, 12, 12, 0, 0, 16, 16, 16, 0, 0, 0, 1, 2, -100]


In [15]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["text"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["upos"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [16]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map: 100%|██████████| 138/138 [00:00<00:00, 10220.26 examples/s]


In [17]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
print(batch["labels"])
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,   11,   12,   12,   12,    0,    0,   16,   16,   16,    0,    0,
            0,    1,    2, -100],
        [-100,   11,   12,   14,   14,   14,   14,    0,    0,   16,   16,   16,
            1,    2, -100, -100]])
[-100, 11, 12, 12, 12, 0, 0, 16, 16, 16, 0, 0, 0, 1, 2, -100]
[-100, 11, 12, 14, 14, 14, 14, 0, 0, 16, 16, 16, 1, 2, -100]


In [19]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [20]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

In [21]:
import evaluate

metric = evaluate.load("seqeval")

In [22]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [23]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [24]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [25]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [26]:
from transformers import get_scheduler

num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [27]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [28]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in tqdm(range(num_train_epochs)):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

 50%|█████     | 133/266 [00:30<00:24,  5.48it/s]/home/admin123/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/admin123/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/admin123/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/admin123/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PUNCT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/admin123/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be 

epoch 0: {'precision': 0.826133909287257, 'recall': 0.7507360157016683, 'f1': 0.7866323907455013, 'accuracy': 0.8773584905660378}


100%|██████████| 2/2 [01:01<00:00, 30.85s/it]t/s]

epoch 1: {'precision': 0.8488120950323974, 'recall': 0.817047817047817, 'f1': 0.8326271186440678, 'accuracy': 0.9019688269073011}
